# Week Four Exercise: MLPs + Dropout

### Data things

We're doing a sentiment classification task. So first load the Stanford Sentiment Treebank data.

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import re
import random

sst_home = '../data/trees'

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}
    # so labels of 0 and 1 in te 5-wayclassificaiton are 0 in the 2-way. 3 and 4 are 1, and 2 is none
    # because we don't have a neautral class. 

PADDING = "<PAD>"
UNKNOWN = "<UNK>"
max_seq_length = 20

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            example['label'] = easy_label_map[int(line[1])]
            if example['label'] is None:
                continue
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data
     
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

And extract bag-of-words feature vectors.

In [2]:
import collections
import numpy as np

def tokenize(string):
    return string.split()

def build_dictionary(training_datasets):
    """
    Extract vocabulary and build dictionary.
    """  
    word_counter = collections.Counter()
    for i, dataset in enumerate(training_datasets):
        for example in dataset:
            word_counter.update(tokenize(example['text']))
        
    vocabulary = set([word for word in word_counter])
    vocabulary = list(vocabulary)
    vocabulary = [PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))

    return word_indices, len(vocabulary)

def sentences_to_padded_index_sequences(word_indices, datasets):
    """
    Annotate datasets with feature vectors. Adding right-sided padding. 
    """
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['text_index_sequence'] = torch.zeros(max_seq_length)

            token_sequence = tokenize(example['text'])
            padding = max_seq_length - len(token_sequence)

            for i in range(max_seq_length):
                if i >= len(token_sequence):
                    index = word_indices[PADDING]
                else:
                    if token_sequence[i] in word_indices:
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                example['text_index_sequence'][i] = index

            example['text_index_sequence'] = example['text_index_sequence'].long().view(1,-1)
            example['label'] = torch.LongTensor([example['label']])


word_to_ix, vocab_size = build_dictionary([training_set])
sentences_to_padded_index_sequences(word_to_ix, [training_set, dev_set, test_set])

We want to feed data to our model in mini-batches so we need a data iterator that will "batchify" the data.

In [3]:
# This is the iterator we'll use during training. 
# It's a generator that gives you one batch at a time.
def data_iter(source, batch_size):
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)   
        batch_indices = order[start:start + batch_size]
        yield [source[index] for index in batch_indices]

# This is the iterator we use when we're evaluating our model. 
# It gives a list of batches that you can then iterate through.
def eval_iter(source, batch_size):
    batches = []
    dataset_size = len(source)
    start = -1 * batch_size
    order = range(dataset_size)
    random.shuffle(order)

    while start < dataset_size - batch_size:
        start += batch_size
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        batches.append(batch)
        
    return batches

# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    labels = []
    for dict in batch:
        vectors.append(dict["text_index_sequence"])
        labels.append(dict["label"])
    return vectors, labels


### Model time!

We need to define an evaluation function,

In [4]:
def evaluate(model, data_iter):
    model.eval()
    correct = 0
    total = 0
    for i in range(len(data_iter)):
        vectors, labels = get_batch(data_iter[i])
        vectors = Variable(torch.stack(vectors).squeeze())
        labels = torch.stack(labels).squeeze()
        output = model(vectors)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    return correct / float(total)

Now for the fun part!

**Part One:** <br>
Embed your word indices into a high dimensional vector space. For each word, we want a 100-D word vector. So define a randomly initialized matrix of word embeddings (take a look at `nn.Embedding`). Next, define a model that takes the word embeddings, adds them up, and passes that vector to an MLP with two ReLU hidden layers of 50 dimensions each. 

Remember that you need to initalize your embedding and linear layers! Do a uniform initialization around 0 from [-0.1, 0.1].


If your model works, it should be able to overfit, reaching about 90% accuracy *on the training set* in the first 500 steps.

**Part Two:** <br>
After the embedding layer and after the final affine layer, add dropout with a 50% drop rate.

Remember that dropout behaves differently at training time and at test time. So we need to put our model in _train_ mode when training, and _eval_ mode when evaluating/testing.

If dropout works, your model should overfit less, but should still perform about as well (or, hopefully, better) on the dev set.

In [16]:
# A Multi-Layer Perceptron (MLP)
class MLPClassifier(nn.Module): # inheriting from nn.Module!
    
    def __init__(self, input_size, embedding_dim, hidden_dim, num_labels):
        super(MLPClassifier, self).__init__()
        
        # Define the parameters that you will need.  
        # You need an embedding matrix, parameters for affine mappings and ReLus
        # Pay attention to dimensions!
        
        """
        Your code goes here!
        """
        
    def forward(self, x):
        # Pass the input through your layers in order
         """
        Your code goes here!
        """
        return

    def init_weights(self):
        """
        Your code goes here!
        """

We now define our training loop,

In [17]:
def training_loop(model, loss, optimizer, training_iter, dev_iter, train_eval_iter):
    step = 0
    for i in range(num_train_steps):
        model.train()
        vectors, labels = get_batch(next(training_iter))
        vectors = Variable(torch.stack(vectors).squeeze())
        labels = Variable(torch.stack(labels).squeeze())

        model.zero_grad()
        output = model(vectors)

        lossy = loss(output, labels)
        lossy.backward()
        optimizer.step()

        if step % 100 == 0:
            print( "Step %i; Loss %f; Train acc: %f; Dev acc %f" 
                %(step, lossy.data[0], evaluate(model, train_eval_iter), evaluate(model, dev_iter)))

        step += 1

Let's define our hyperparameters

In [21]:
# Hyper Parameters 
"""
Fill in the blank!
"""
input_size = # ??
num_labels = # ??
hidden_dim = # ??
embedding_dim = # ??
batch_size = 32
learning_rate = 0.004
num_train_steps = 1000

Finally, we can build and train our model!

In [22]:
model = MLPClassifier(input_size, embedding_dim, hidden_dim, num_labels)
    
# Loss and Optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
training_iter = data_iter(training_set, batch_size)
train_eval_iter = eval_iter(training_set[0:500], batch_size)
dev_iter = eval_iter(dev_set[0:500], batch_size)

training_loop(model, loss, optimizer, training_iter, dev_iter, train_eval_iter)

Step 0; Loss 0.703077; Train acc: 0.466000; Dev acc 0.460000
Step 100; Loss 0.620177; Train acc: 0.796000; Dev acc 0.730000
Step 200; Loss 0.517760; Train acc: 0.878000; Dev acc 0.746000
Step 300; Loss 0.310241; Train acc: 0.918000; Dev acc 0.760000
Step 400; Loss 0.312887; Train acc: 0.944000; Dev acc 0.756000
Step 500; Loss 0.158119; Train acc: 0.962000; Dev acc 0.736000
Step 600; Loss 0.286196; Train acc: 0.968000; Dev acc 0.776000
Step 700; Loss 0.010323; Train acc: 0.984000; Dev acc 0.758000
Step 800; Loss 0.055903; Train acc: 0.986000; Dev acc 0.772000
Step 900; Loss 0.050397; Train acc: 0.990000; Dev acc 0.790000


Let's see how it performs on the held out test set,

In [24]:
# Test the model
test_iter = eval_iter(test_set, batch_size)
test_acc = evaluate(model, test_iter)
print('Accuracy of the network on the test data: %f' % (100 * test_acc))

Accuracy of the network on the test data: 76.551345


<br>
Why isn't the model performancy very good? What could we do to make it better?